# Geração das respostas e métricas com o Gemini

In [10]:
import pandas as pd

df = pd.read_csv(
    '../../data/inference_data.csv',
    encoding='utf-8',
    sep=','
)

# Converter register_time e start_time para datetime se não estiverem
df['register_time'] = pd.to_datetime(df['register_time'])
df['start_time'] = pd.to_datetime(df['start_time'])

# Verificar se o dataset está ordenado por register_time e start_time
print("=== VERIFICANDO ORDENAÇÃO DO DATASET ===")
print(f"Total de registros: {len(df)}")
print(f"Primeiros 5 registros (register_time):")
print(df[['register_time', 'start_time', 'title']].head(10))

# Verificar se está ordenado primeiro por register_time e depois por start_time
is_sorted_register = df['register_time'].is_monotonic_increasing
is_sorted_both = df.sort_values(['register_time', 'start_time']).equals(df)

print(f"\nDataset está ordenado apenas por register_time? {is_sorted_register}")
print(f"Dataset está ordenado por register_time E start_time? {is_sorted_both}")

# Se não estiver ordenado, vamos identificar onde estão os problemas
if not is_sorted_both:
    print("\n=== PROBLEMAS DE ORDENAÇÃO DETECTADOS ===")

    # Encontrar onde a ordem está quebrada
    problems = []
    for i in range(1, len(df)):
        current_register = df.iloc[i]['register_time']
        previous_register = df.iloc[i-1]['register_time']
        current_start = df.iloc[i]['start_time']
        previous_start = df.iloc[i-1]['start_time']

        # Verificar se há problema na ordenação
        if (current_register < previous_register) or \
           (current_register == previous_register and current_start < previous_start):
            problems.append({
                'index': i,
                'current_register_time': current_register,
                'previous_register_time': previous_register,
                'current_start_time': current_start,
                'previous_start_time': previous_start,
                'current_task': df.iloc[i]['title'],
                'previous_task': df.iloc[i-1]['title']
            })

    print(f"Encontrados {len(problems)} problemas de ordenação:")
    for problem in problems[:10]:  # Mostrar apenas os primeiros 10
        print(f"  Linha {problem['index']}: {problem['current_register_time']} / {problem['current_start_time']}")
        print(f"    vs anterior: {problem['previous_register_time']} / {problem['previous_start_time']}")
        print(f"    Tarefa atual: {problem['current_task']}")
        print(f"    Tarefa anterior: {problem['previous_task']}")
        print()


df = df.sort_values(['register_time', 'start_time']).reset_index(drop=True)

print(f"\nPrimeiros 5 registros após verificação/ordenação:")
print(df[['register_time', 'start_time', 'title']].head(20))

df.info()

=== VERIFICANDO ORDENAÇÃO DO DATASET ===
Total de registros: 143
Primeiros 5 registros (register_time):
        register_time          start_time                             title
0 2017-11-27 06:58:00 2017-12-14 23:00:00      Attend the school graduation
1 2018-08-07 11:25:00 2018-08-10 22:00:00             Go to Baile da Gaiola
2 2020-09-18 00:58:00 2020-09-18 08:00:00  Study calculus and problem set 2
3 2021-05-18 21:23:00 2021-05-19 19:00:00    Go to psychiatrist appointment
4 2021-05-21 02:39:00 2021-05-21 15:30:00          Facetime with Alessandra
5 2021-07-07 23:53:00 2021-07-08 11:00:00                       Take exam 3
6 2021-09-01 15:56:00 2021-09-03 16:00:00    Go to psychologist appointment
7 2021-10-25 11:01:00 2021-11-01 19:30:00            Consult with Dr. Bruno
8 2022-01-17 13:36:00 2022-01-17 17:55:00                     Go to the gym
9 2022-01-20 06:38:00 2022-01-20 10:45:00                 Have chiropractic

Dataset está ordenado apenas por register_time? False
Datas

In [11]:
# Converter blocos de 30 minutos (336 blocos) para blocos de 1 hora (168 blocos)
print("=== CONVERTENDO BLOCOS DE 30 MIN PARA 1 HORA ===")

# Verificar valores originais da coluna start_time_slot
print(f"Valores originais - Min: {df['start_time_slot'].min()}, Max: {df['start_time_slot'].max()}")
print(f"Total de valores únicos originais: {df['start_time_slot'].nunique()}")

# Fórmula de conversão:
# - Blocos de 30 min: 1-336 (segunda 00:00 até domingo 00:00)
# - Blocos de 1 hora: 1-168 (segunda 00:00 até domingo 00:00)
# - Conversão: bloco_1h = ceil(bloco_30min / 2)

import math

# Aplicar a conversão
df['start_time_slot_1h'] = df['start_time_slot'].apply(lambda x: math.floor(x / 2))

# Verificar resultados da conversão
print(f"\nValores convertidos - Min: {df['start_time_slot_1h'].min()}, Max: {df['start_time_slot_1h'].max()}")
print(f"Total de valores únicos convertidos: {df['start_time_slot_1h'].nunique()}")

# Substituir a coluna original pela convertida
df['start_time_slot'] = df['start_time_slot_1h']
df = df.drop('start_time_slot_1h', axis=1)

print(f"\n✅ Conversão concluída!")
print(f"Coluna start_time_slot agora contém blocos de 1 hora (1-168)")

# Mostrar alguns exemplos da conversão
print(f"\nExemplos dos primeiros valores convertidos:")
print(df[['title', 'start_time_slot']].head(10))

=== CONVERTENDO BLOCOS DE 30 MIN PARA 1 HORA ===
Valores originais - Min: 6, Max: 326
Total de valores únicos originais: 92

Valores convertidos - Min: 3, Max: 163
Total de valores únicos convertidos: 73

✅ Conversão concluída!
Coluna start_time_slot agora contém blocos de 1 hora (1-168)

Exemplos dos primeiros valores convertidos:
                              title  start_time_slot
0      Attend the school graduation               95
1             Go to Baile da Gaiola              118
2  Study calculus and problem set 2              104
3    Go to psychiatrist appointment               67
4          Facetime with Alessandra              111
5                       Take exam 3               83
6    Go to psychologist appointment              112
7            Consult with Dr. Bruno               19
8                     Go to the gym               17
9                 Have chiropractic               82


In [12]:
def get_week_context_tasks(df: pd.DataFrame, current_index: int) -> list:
    """
    Para uma tarefa específica (current_index), retorna todas as tarefas da mesma semana ISO
    e de semanas anteriores como contexto.

    Args:
        df (pd.DataFrame): DataFrame com as tarefas ordenadas por register_time
        current_index (int): Índice da tarefa atual

    Returns:
        list: Lista de dicionários com tarefas da mesma semana e anteriores
    """

    # Obter a tarefa atual
    current_task = df.iloc[current_index]

    # Extrair ano e semana ISO da data de registro da tarefa atual
    current_register_time = current_task['register_time']
    current_start_year = current_task['start_iso_year']  # Ano ISO
    current_start_week = current_task['start_iso_week']  # Semana ISO

    context_tasks = []
    for _, task in df.iterrows():
        task_register_time = task['register_time']
        task_start_year = task['start_iso_year']
        task_start_week = task['start_iso_week']
        task_week_register_sequence = task['week_register_sequence']

        # Incluir se:
        # Uma tarefa é do mesmo ano e semana e foi registrada antes ou no mesmo momento que a tarefa atual
        if ((task_start_year == current_start_year) and
            (task_start_week == current_start_week) and
            (task_register_time <= current_register_time) and
            (task_week_register_sequence < current_task['week_register_sequence'])):

            # # Pular a própria tarefa atual
            if task.name == current_index:
                continue

            context_task = {
                'email_address': task['email_address'],
                'title': task['title'],
                'duration_minute': task['duration_minute'],
                'start_time': task['start_time'],
                'start_time_slot': task['start_time_slot']
            }
            context_tasks.append(context_task)

    return context_tasks


In [13]:
def get_month_context_tasks(df: pd.DataFrame, current_index: int) -> list:
    """
    Para uma tarefa específica (current_index), retorna todas as tarefas do mesmo mês como contexto, baseado na coluna start_time.

    Args:
        df (pd.DataFrame): DataFrame com as tarefas ordenadas por register_time
        current_index (int): Índice da tarefa atual

    Returns:
        list: Lista de dicionários com tarefas do mesmo mês e anteriores
    """

    # Obter a tarefa atual
    current_task = df.iloc[current_index]

    # Extrair ano e mês da data de início da tarefa atual
    current_register_time = current_task['register_time']
    current_start_time = pd.to_datetime(current_task['start_time'])
    current_year = current_start_time.year
    current_month = current_start_time.month

    context_tasks = []
    for _, task in df.iterrows():
        task_register_time = task['register_time']
        task_start_time = pd.to_datetime(task['start_time'])
        task_year = task_start_time.year
        task_month = task_start_time.month

        # Incluir se:
        # 1. A tarefa é do mesmo ano e mês e foi registrada antes ou no mesmo momento que a tarefa atual
        is_same_month_and_earlier = (
            (task_year == current_year) and
            (task_month == current_month) and
            (task_register_time <= current_register_time) and
            (task_start_time < current_start_time)
        )

        if is_same_month_and_earlier:
            # Pular a própria tarefa atual
            if task.name == current_index:
                continue

            context_task = {
                'email_address': task['email_address'],
                'title': task['title'],
                'duration_minute': task['duration_minute'],
                'start_time': task['start_time'],
                'start_time_slot': task['start_time_slot']
            }
            context_tasks.append(context_task)

    return context_tasks

In [14]:
def get_10_last_tasks(df: pd.DataFrame, current_index: int) -> list:
    """
    Para uma tarefa específica (current_index), retorna as 10 tarefas anteriores como contexto.

    Args
        df (pd.DataFrame): DataFrame com as tarefas ordenadas por register_time
        current_index (int): Índice da tarefa atual

    Returns:
        list: Lista de dicionários com as 10 tarefas anteriores
    """

    current_task = df.iloc[current_index]
    current_register_time = current_task['register_time']
    current_start_time = pd.to_datetime(current_task['start_time'])

    context_tasks = []
    for i in range(current_index - 1, max(-1, current_index - 11), -1):
        if i < 0:
            break

        task = df.iloc[i]
        task_register_time = task['register_time']
        task_start_time = pd.to_datetime(task['start_time'])

        context_task = {
            'email_address': task['email_address'],
            'title': task['title'],
            'duration_minute': task['duration_minute'],
            'start_time': task['start_time'],
            'start_time_slot': task['start_time_slot']
        }
        context_tasks.append(context_task)

    return context_tasks


In [16]:
import yaml

yaml_path = '../../scheduler.prompt.yml'
with open(yaml_path, 'r', encoding='utf-8') as file:
    verbalizer_config = yaml.safe_load(file)

# Acessar partes específicas do YAML
messages = verbalizer_config.get('messages', [])
model = verbalizer_config.get('model', '')
model_params = verbalizer_config.get('modelParameters', {})
temperature = verbalizer_config.get('modelParameters').get('temperature')
max_tokens = verbalizer_config.get('modelParameters').get('max_tokens')


print(f"Modelo: {model}")
print(f"Parâmetros do modelo: {model_params}")
print(f'Temperatura: {temperature}')
print(f'Máximo de tokens: {max_tokens}')

# Exemplo de como acessar o conteúdo das mensagens
print('Mensagens:')
print(f'System: {messages[0].get("content", "")}')
print(f'User: {messages[1].get("content", "")}')

Modelo: openai/gpt-4o
Parâmetros do modelo: {'max_tokens': 2048, 'temperature': 0.0, 'top_p': 0.8}
Temperatura: 0.0
Máximo de tokens: 2048
Mensagens:
System: Pense passo a passo como um especialista em agendamento de tarefas. Você receberá 6 colunas de atributos das tarefas anteriores a do agendamento, chamada tarefas de contexto. Abaixo estão as colunas das tarefas de contexto:
1. **Colunas**: - Coluna 1 **email_address**: Email do usuário que está agendando a tarefa - Coluna 2 **title**: Descrição da tarefa - Coluna 3 **duration_minute**: Duração estimada da tarefa (em minutos) - Coluna 4 **start_time**: Data de início da tarefa (formato ISO 8601) - Coluna 5 **start_time_slot**: Bloco de horário ocupado para início da tarefa
Cada coluna descreve um atributo da tarefa a ser agendada.
Além disso, você receberá uma nova tarefa a ser agendada, que contém as seguintes colunas: - Coluna 1 **email_address**: Email do usuário que está agendando - Coluna 2 **title**: Descrição da tarefa - Col

In [17]:
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')
)

In [ ]:
import numpy as np

def schedule_task(task: dict, context_tasks: list) -> str:
    """
    Agendar uma tarefa com base no contexto de tarefas anteriores.

    Args:
        task (dict): Dicionário com os detalhes da tarefa a ser agendada.
        context_tasks (list): Lista de dicionários com as tarefas anteriores como contexto.

    Returns:
        str: Resposta do modelo sobre o agendamento da tarefa.
    """

    response = client.chat.completions.create(
        model='gpt-4.1',
        messages=[
            {
                "role": "system",
                "content": messages[0]['content']
                .replace('{{PREVIOUS_TASKS}}', str(context_tasks))
            },
            {
                "role": "user",
                "content": messages[1]['content']
                .replace('{{TASK}}', str(task))
            },
        ],
        temperature=temperature,
        logprobs=True,
        top_logprobs=20,
        seed=22,
    )

    return response


=== AGENDANDO TAREFA ===
Tarefa a ser agendada: 9: {'email_address': 'sample@email.com', 'title': 'Have chiropractic', 'duration_minute': 120}
Contexto de tarefas anteriores:
  1. Go to the gym (Slot: 17, Duração: 60min)

=== RESPOSTA DO MODELO ===
Resposta do modelo: 18


In [19]:
def extract_top_blocks_from_response(response, top_k=5):
    """
    Extrai os top K blocos de horário das logprobs da resposta do modelo.

    Args:
        response: Resposta do modelo OpenAI com logprobs
        top_k: Número de top blocos a extrair (padrão: 5)

    Returns:
        list: Lista de dicionários com informações completas dos top K blocos
              Formato: [{'block': int, 'probability': float, 'rank': int, 'token': str}, ...]
              Ordenados por probabilidade (maior para menor)
    """
    try:
        logprobs = response.choices[0].logprobs.content[0].top_logprobs

        # Extrair os blocos com todas as informações
        blocks_data = []
        for i, logprob in enumerate(logprobs):
            if len(blocks_data) >= top_k:
                break

            try:
                block_number = int(logprob.token) - 1
                probability = np.exp(logprob.logprob)

                block_info = {
                    'block': block_number,
                    'probability': probability,
                    'rank': i + 1,
                    'token': logprob.token,
                    'logprob': logprob.logprob
                }
                blocks_data.append(block_info)

            except ValueError:
                print(f"Token inválido encontrado: {logprob.token}")
                continue

        # Retornar apenas os top K blocos válidos (que são números)
        valid_blocks = [block for block in blocks_data if block.get('block') is not None]

        return valid_blocks[:top_k]

    except Exception as e:
        print(f"Erro ao extrair blocos: {e}")
        return []


In [ ]:
def extract_response(response):
    # Extrair informações relevantes da resposta da API
    try:
        return response.choices[0].message.content
    except (KeyError, IndexError):
        return "Erro ao extrair resposta"


In [ ]:
def extract_tokens_spent_from_response(response):
    ''''
    Extrai os tokens de saída da resposta do modelo.
    Args:
        response: Resposta do modelo OpenAI
    Returns:
        int: Número de tokens gastos em todo o processo
    '''
    return response.usage.total_tokens

=== EXTRAINDO TOKENS GASTOS ===
Tokens gastos: 543


In [22]:
def calculate_recall_metrics(true_block, predicted_blocks_data):
    """
    Calcula as métricas Recall@1 e Recall@5.

    Args:
        true_block (int): Bloco de horário original/correto
        predicted_blocks_data (list): Lista de dicionários com informações dos blocos preditos ou lista simples de números dos blocos

    Returns:
        tuple: (recall_at_1, recall_at_5)
    """
    print('=== CÁLCULO DAS MÉTRICAS DE RECALL ===')
    for block in predicted_blocks_data:
        print(f"  - {block['block']} (Prob: {block['probability']:.4f}, Rank: {block['rank']}, Token: {block['token']})")

    # Suportar tanto a estrutura nova quanto a antiga
    if (isinstance(predicted_blocks_data, list)) and (isinstance(predicted_blocks_data[0], dict)) and len(predicted_blocks_data) > 0:
        # Nova estrutura: extrair apenas os números dos blocos
        predicted_blocks = [block['block'] for block in predicted_blocks_data if block.get('block') is not None]
    else:
        # Estrutura antiga: lista simples de números
        predicted_blocks = predicted_blocks_data
        return 0, 0

    # Recall@1: o bloco correto está na primeira posição?
    if (predicted_blocks[0] == true_block):
        recall_at_1 = True
    else:
        recall_at_1 = False

    # Recall@5: o bloco correto está entre os 5 primeiros?
    if true_block in predicted_blocks:
        recall_at_5 = True
    else:
        recall_at_5 = False

    return recall_at_1, recall_at_5


In [ ]:
import time

def inference_loop(df: pd.DataFrame, mode:str='week'):
    # Criar DataFrame vazio para armazenar os resultados
    results_df = pd.DataFrame(columns=[
        'task_index', 'original_time_slot', 'predicted_time_slot', 'recall_at_1', 'recall_at_5', 'tokens_spent'
    ])

    # Loop de inferência para todas as tarefas
    for _, task in df.iterrows():
        if mode == 'week':
            context_tasks = get_week_context_tasks(df, task.name)
        elif mode == 'month':
            context_tasks = get_month_context_tasks(df, task.name)
        else:
            context_tasks = get_10_last_tasks(df, task.name)

        print(f'Contexto de tarefas para a tarefa {task.name}:')
        print(f'{task['title']}, Duração: {task['duration_minute']} minutos, Slot: {task['start_time_slot']}')
        for i, context_task in enumerate(context_tasks):
            print(f"  - Tarefa {i}: {context_task}")

        filtered_task = {
            'email_address': task['email_address'],
            'title': task['title'],
            'duration_minute': task['duration_minute'],
        }
        try:
            time.sleep(1)  # Small delay to respect API rate limits
            response = schedule_task(filtered_task, context_tasks)
        except Exception as e:
            new_row = pd.DataFrame({
                'task_index': [task.name],
                'original_time_slot': [task['start_time_slot']],
                'predicted_time_slot': None,
                'recall_at_1': None,
                'recall_at_5': None,
                'tokens_spent': None
            })
            results_df = pd.concat([results_df, new_row], ignore_index=True)

            print(f"Erro ao agendar tarefa: {e}")
            continue

        top_5_blocks_data = extract_top_blocks_from_response(response, top_k=5)
        block_data = int(extract_response(response)) - 1

        recall_1, recall_5 = calculate_recall_metrics(task['start_time_slot'], top_5_blocks_data)
        print(f'Time Slot original: {task['start_time_slot']}')
        print(f'Time Slot predito: {block_data}')
        print(f'Recall@1: {'✅' if recall_1 else '❌'}')
        print(f'Recall@5: {'✅' if recall_5 else '❌'}')

        # Adicionar resultados ao DataFrame
        new_row = pd.DataFrame({
            'task_index': [task.name],
            'original_time_slot': [task['start_time_slot']],
            'predicted_time_slot': [block_data],
            'recall_at_1': [recall_1],
            'recall_at_5': [recall_5],
            'tokens_spent': [extract_tokens_spent_from_response(response)]
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)

    return results_df

results_csv_path = '../../data/'

week_results_df = inference_loop(df, mode='week')
week_results_df.to_csv(results_csv_path + 'week_scheduling_results.csv', index=False)
print('Resultados da semana salvos em week_scheduling_results.csv')

month_results_df = inference_loop(df, mode='month')
month_results_df.to_csv(results_csv_path + 'month_scheduling_results.csv', index=False)
print('Resultados do mês salvos em month_scheduling_results.csv')

ten_last_results_df = inference_loop(df, mode='10_last')
ten_last_results_df.to_csv(results_csv_path + '10_last_scheduling_results.csv', index=False)
print('Resultados das 10 últimas tarefas salvos em 10_last_scheduling_results.csv')


Contexto de tarefas para a tarefa 0:
Attend the school graduation, Duração: 480 minutos, Slot: 95
=== CÁLCULO DAS MÉTRICAS DE RECALL ===
  - 0 (Prob: 0.5066, Rank: 1, Token: 1)
  - 8 (Prob: 0.3073, Rank: 2, Token: 9)
  - 48 (Prob: 0.0777, Rank: 3, Token: 49)
  - 24 (Prob: 0.0367, Rank: 4, Token: 25)
  - 32 (Prob: 0.0223, Rank: 5, Token: 33)
Time Slot original: 95
Time Slot predito: 0
Recall@1: ❌
Recall@5: ❌
Contexto de tarefas para a tarefa 1:
Go to Baile da Gaiola, Duração: 480 minutos, Slot: 118
=== CÁLCULO DAS MÉTRICAS DE RECALL ===
  - 0 (Prob: 0.6043, Rank: 1, Token: 1)
  - 24 (Prob: 0.1731, Rank: 2, Token: 25)
  - 48 (Prob: 0.0341, Rank: 3, Token: 49)
  - 18 (Prob: 0.0265, Rank: 4, Token: 19)
  - 72 (Prob: 0.0265, Rank: 5, Token: 73)
Time Slot original: 118
Time Slot predito: 0
Recall@1: ❌
Recall@5: ❌
Contexto de tarefas para a tarefa 2:
Study calculus and problem set 2, Duração: 300 minutos, Slot: 104
=== CÁLCULO DAS MÉTRICAS DE RECALL ===
  - 0 (Prob: 0.9915, Rank: 1, Token: 1)

KeyboardInterrupt: 